In [19]:
import os
import nest_asyncio

from llama_index.core import Document, VectorStoreIndex, StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.vector_stores import VectorStoreQueryResult
from qdrant_client import QdrantClient, AsyncQdrantClient
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core import Settings
from typing import List
from dotenv import load_dotenv
import json
import os
nest_asyncio.apply()
load_dotenv()

True

## Setup function to transform JSON into documents.

In [20]:
def create_faq_docs(json_path: str, documents: List[Document]) -> int:
    initial_count = len(documents)  # Track initial number of documents
    
    with open(json_path, mode="r", encoding="utf-8") as file:
        faq_data = json.load(file)
    
    for i, entry in enumerate(faq_data):
        category = entry.get("category", "General").strip()
        question = entry.get("question", "").strip()
        answer = entry.get("answer", "").strip()
        
        if question and answer:
            json_data = {
                "Question": question,
                "Answer": answer,
            }
            
            metadata = {
                "category": category,
                "language": "English",
            }
            
            text_content = json.dumps(json_data, ensure_ascii=False)
            document = Document(text=text_content, metadata=metadata, doc_id=f"faq_{i}")
            documents.append(document)
    
    # Return the number of documents appended
    return len(documents) - initial_count

In [21]:
def print_documents_by_indices(documents: List[Document], indices: List[int]) -> None:
    """
    Print documents at specific indices.

    Parameters:
        documents (List[Document]): The list of documents.
        indices (List[int]): A list of indices of the documents to print.
    """
    for index in indices:
        if 0 <= index < len(documents):  # Check if index is within bounds
            print(f"Document {index}:")
            print(f"  doc_id: {documents[index].doc_id}")
            print(f"  text: {documents[index].text}")
            print(f"  metadata: {documents[index].metadata}")
            print()  # Add a blank line for readability
        else:
            print(f"Index {index} is out of bounds. Please specify a valid index.")

### Show some documents example.

In [22]:
# Example usage:
documents = []
faq_count = create_faq_docs("documents/loa_cpaxtra_faq.json", documents)

# Specify the list of indices to print
print("Example documents:")
indices_to_print = [0, 10]
print_documents_by_indices(documents, indices_to_print)
print(f"{faq_count} FAQ documents have been appended.")

Example documents:
Document 0:
  doc_id: faq_0
  text: {"Question": "DoA/LoA ต่างกันอย่างไร เมื่อไรอ้างอิง DoA หรือ LoA", "Answer": "LOA (Level of Authorization) คือระดับอำนาจในการอนุมัติภายในบริษัท โดยมีตารางกำหนดวงเงินตามตำแหน่งในเอกสาร CPAX-FN-001 ส่วน DoA (Delegation of Authority) เป็นการมอบอำนาจในบริบทที่กว้างกว่า เช่น การตัดสินใจเชิงกลยุทธ์หรือวงเงินที่เกินจาก LOA โดยต้องได้รับอนุมัติจากกรรมการหรือผู้มีอำนาจระดับสูง"}
  metadata: {'category': 'General', 'language': 'English'}

Document 10:
  doc_id: faq_10
  text: {"Question": "วงเงินผู้มีอำนาจอนุมัติ Purchase Requisition ในระบบ Oracle ERP", "Answer": "ระบบ Oracle ERP มีการตั้งค่าระดับการอนุมัติ PR ตามโครงสร้างอำนาจที่ระบุไว้ใน LoA เช่น หัวหน้าฝ่ายอาจอนุมัติได้ไม่เกิน 500,000 บาท ขณะที่ผู้บริหารระดับสูง เช่น CFO หรือ CEO สามารถอนุมัติวงเงินที่สูงขึ้น ซึ่งต้องมีการอ้างอิงตามเอกสาร LoA และ SEA ถ้ามี."}
  metadata: {'category': 'LoA Policy', 'language': 'English'}

25 FAQ documents have been appended.


## Setup Cohear Embedding service

In [23]:
embed_model = CohereEmbedding(
    api_key=os.getenv("COHEAR_API_KEY"),
    model_name=os.getenv("COHEAR_MODEL_ID"),
    input_type="search_document",
    embedding_type="float",
)

Settings.chunk_size = 512

## Innitiates VectorStore database (Qdrant)

In [24]:
# creates a persistant index to disk
client = QdrantClient(url="http://localhost:6334", api_key=os.getenv("QDRANT_API_KEY"),  prefer_grpc=True)

# # delete collection if it exists
if client.collection_exists(os.getenv("QDANT_COLLENCTION_NAME")):
    client.delete_collection(os.getenv("QDANT_COLLENCTION_NAME"))

# create our vector store with hybrid indexing enabled
vector_store = QdrantVectorStore(
    os.getenv("QDANT_COLLENCTION_NAME"),
    client=client,
    enable_hybrid=True,
    batch_size=20,
    prefer_grpc=True,
)

/var/folders/z5/qxv1ygl57v34c8nqkcj1m3200000gn/T/ipykernel_66472/55195034.py:2: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(url="http://localhost:6334", api_key=os.getenv("QDRANT_API_KEY"),  prefer_grpc=True)


## Start embedding process.... into vector database

In [29]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents=documents, embed_model=embed_model, storage_context=storage_context,
)


## Try to retrive relavent nodes with question.

In [30]:
embed_model = CohereEmbedding(
    api_key=os.getenv("COHEAR_API_KEY"),
    model_name=os.getenv("COHEAR_MODEL_ID"),
    input_type="search_query",
    embedding_type="float",
)

search_query_retriever = index.as_retriever()

search_query_retrieved_nodes = search_query_retriever.retrieve(
"Do all Walmart locations offer scan & go?"
)

In [27]:
from llama_index.core.response.notebook_utils import display_source_node
for n in search_query_retrieved_nodes:
    display_source_node(n, source_length=2000)

**Node ID:** 2c5851db-8213-45de-aa4d-7382638846de<br>**Similarity:** 0.0957329124212265<br>**Text:** {"Question": "การพิจารณาว่าจำเป็นต้องเซ็นสัญญาหรือไม่", "Answer": "ขึ้นอยู่กับประเภทและมูลค่าของการดำเนินการ เช่น กรณีซื้อขาย/ให้บริการกับบุคคลภายนอก หรือกรณีมีข้อผูกพันทางกฎหมาย จำเป็นต้องทำสัญญา โดยฝ่ายจัดซื้อ/กฎหมายสามารถช่วยประเมินว่ากรณีใดจำเป็นต้องมีสัญญา."}<br>

**Node ID:** 6e5ac63e-37bb-4f75-80b5-c066eabca601<br>**Similarity:** 0.0618140883743763<br>**Text:** {"Question": "วงเงินผู้มีอำนาจอนมัติ PO manual", "Answer": "การออก PO แบบ manual ต้องได้รับการอนุมัติตามระดับวงเงินที่กำหนดใน LoA ซึ่งระบุว่าใครสามารถอนุมัติได้ในแต่ละระดับราคา และต้องมีเหตุผลรองรับว่าทำไมจึงไม่ใช้ระบบ Oracle ERP ในการออก PR/PO ตามปกติ เช่น กรณีฉุกเฉินหรือข้อจำกัดจาก vendor."}<br>